In [1]:
# Importing required libraries

import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary


from pymongo import MongoClient

In [2]:
# Creating a connection to MongoDB
client = MongoClient('192.168.30.114', 27017)
db = client['news']
collection = db['elespectador']

In [3]:
# Base URL of the site to be analyzed
SITE_URL = 'https://www.elespectador.com'

In [4]:
# Firefox web driver path
# Download the driver for you S.O. here: https://github.com/mozilla/geckodriver/releases
options = Options()
#options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
options.headless = True
options.binary_location = '/usr/bin/firefox'
DRIVER_PATH = './geckodriver'
#DRIVER_PATH = 'I:\Escritorio\geckodriver\geckodriver.exe'

In [5]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH, options=options)

In [6]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [8]:
news = []
blocks = []
for i in range(10):
    soup = make_request(browser, f'/archivo/politica/{i + 2}')
    layout = soup.find(class_ = 'Layout-flexAds')
    blocks = layout.find_all('section')
    #print(len(blocks))
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card') 
    #print(len(cards))
    
    for card in cards:
        news.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })
    
news
    #print(f"Soy el numero {i + 2}")

[{'title': '“Algunos quieren sacar provecho de la visita de Blinken”: Duque a Vivanco ',
  'relative_path': '/politica/algunos-quieren-sacar-provecho-de-la-visita-de-blinken-duque-a-vivanco/'},
 {'title': 'Petro tomará “acciones jurídicas” contra exgeneral venezolano Hugo Carvajal ',
  'relative_path': '/politica/petro-tomara-acciones-juridicas-contra-exgeneral-venezolano-hugo-carvajal/'},
 {'title': 'Cuando las mujeres se juntaron en el Congreso para ser una sola voz',
  'relative_path': '/politica/cuando-las-mujeres-se-juntaron-en-el-congreso-para-ser-una-sola-voz/'},
 {'title': 'Pandora Papers: oposición citó a altos funcionarios a debate de control político ',
  'relative_path': '/politica/pandora-papers-oposicion-cito-a-altos-funcionarios-a-debate-de-control-politico/'},
 {'title': 'Polémico audio acentúa diferencias entre Cabal y bancada uribista',
  'relative_path': '/politica/polemico-audio-acentua-diferencias-entre-cabal-y-bancada-uribista/'},
 {'title': 'Capturados por presun

In [ ]:
# Getting HTML content for news listing page
soup = make_request(browser, '/archivo/politica/')

In [ ]:
# Finding the section where news are contained 
layout = soup.find(class_ = 'Layout-flexAds')

In [ ]:
# Getting blocks from layout
blocks = layout.find('section').find_all(recursive = False)
print(len(blocks)) # 3 blocks founded, 2 for news and 1 for pagintion

In [ ]:
# Finding and concatenating news cards
cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
len(cards)

In [ ]:
# Building a list with title and relative path of the news founded

news = []

for card in cards:
    news.append({
        'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
        'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
    })

In [ ]:
len(news)

In [ ]:
for n in news:
    # Getting HTML content for news page
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').find('a').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    #n['category'] = soup.find(class_ = 'ACredit-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [ ]:
news

In [ ]:
# Storing extracted information for further analysis
collection.insert_many(news)